**Description**

This script uses post-disaster png satellite images and corresponding json files with post-disaster (damage level) to create one output png image with each pixel labeled as following:

Class 0 - no building or un-classified building

Class 1 - no-damage

Class 2 - minor damage

Class 3 - major damage

Class 4 - destroyed

The script includes and/or makes use of functions created during the xView2 challenge set by DIU (Defense Innovation Unit) through their baseline repository: xView2_baseline.

__[xView2 license](https://github.com/DIUx-xView/xView2_baseline/blob/master/LICENSE.md)__ 

Data need to be in the following format. To create this structure, use **split_into_disasters.py** from __[xView2 baseline](https://github.com/DIUx-xView/xView2_baseline/tree/master)__ :


 folder with the data base: 

├── disaster_name_1

 │------├── images 

 │------│------------<image_id>.png

 │------│------------...

 │------├── labels

 │------│------------<image_id>.json

 │------│------------...

├── disaster_name_2

 │------├── images 

 │------│------------<image_id>.png

 │------│------------...

 │------├── labels

 │------│------------<image_id>.json

 │------│------------...

└── disaster_name_n
 

**Modify path to the data folder with the above structure in the last code block ("path_xBD" variable).**

#### 1. import the necessary libraries

In [22]:
import json
from os import path, walk, makedirs
from sys import exit, stderr

from cv2 import fillPoly, imwrite
import numpy as np
from shapely import wkt
from shapely.geometry import mapping, Polygon
from skimage.io import imread
from tqdm import tqdm


# This removes the massive amount of scikit warnings of "low contrast images"
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

#### 2. Definition of functions

In [23]:
# from XView2 baseline
def get_dimensions(file_path):
    """
    :param file_path: The path of the file 
    :return: returns (width,height,channels)
    """
    # Open the image we are going to mask
    pil_img = imread(file_path)
    img = np.array(pil_img)
    w, h, c = img.shape
    return (w, h, c)

In [24]:
# from xView2 baseline
def save_one_mask(masks, output_path, mask_file_name):
    """
    :param masks: list of masked polygons from the mask_polygons_separately function 
    :param output_path: path to save the masks
    :param mask_file_name: the file name the masks should have 
    """
    # For each filled polygon, write the mask shape out to the file per image
    mask_file_name = path.join(output_path, mask_file_name + '.png')
    imwrite(mask_file_name, masks)

In [25]:
# modified from xView2 baseline 
def mask_polygons_together_with_border(size, shapes_pre, shapes_post, border):
    """
    :param size: A tuple of the (width,height,channels)
    :param shape_pre: A list of points in the polygon from get_feature_info
    :param shape_post: A list of damage level labels from get_feature_damage
    :param border: number of pixels used to shrink the polygon in the case that there is building overlapping

    :returns: an array with the size of the original images, with pixel-wise labels including 
    building and damage information.
    """

    # For each WKT polygon, read the WKT format and fill the polygon as an image
    mask_img_buildings = np.zeros(size, np.uint8)
    mask_img_damage = np.zeros(size, np.uint8)
    
    # declaring colors associated with labels
    buildings_damage_colors = {
        0: (0, 0, 0),      # Class 0 - Black (no building) or un-classified
        1: (255, 255, 255),    # Class 1 - White (no-damage)
        2: (255, 0, 0),    # Class 2 - Red (minor damage)
        3: (0, 255, 0),    # Class 3 - Green (major damage)
        4: (0, 0, 255),    # Class 4 - Blue (destroyed)
    }

    for u in shapes_pre:
        #defining 2 arrays (one for building location, the next for damage level information)
        blank_building =  np.zeros(size, np.uint8)
        blank_damage =  np.zeros(size, np.uint8)

        # Each polygon stored in shapes is a np.ndarray
        poly = shapes_pre[u]
        # the damage label for this particular building
        dam = shapes_post[u]
        # Creating a shapely polygon object out of the numpy array 
        polygon = Polygon(poly)
        
        # Getting the center points from the polygon and the polygon points
        (poly_center_x, poly_center_y) = polygon.centroid.coords[0]
        polygon_points = polygon.exterior.coords

        # Setting a new polygon with each X,Y manipulated based off the center point
        shrunk_polygon = []
        for (x,y) in polygon_points:
            if x < poly_center_x:
                x += border
            elif x > poly_center_x:
                x -= border

            if y < poly_center_y:
                y += border
            elif y > poly_center_y:
                y -= border

            shrunk_polygon.append([x,y])
        
        # Transforming the polygon back to a np.ndarray
        ns_poly = np.array(shrunk_polygon, np.int32)
        
        # depending on level of damage the color (uses a dictionary defined at the top)
        if dam == "no-damage":
            color = buildings_damage_colors[1]
        elif dam == "minor-damage":
            color = buildings_damage_colors[2]
        elif dam == "major-damage":
            color = buildings_damage_colors[3]
        elif dam == "destroyed":           
            color = buildings_damage_colors[4]
        else: # includes 2 cases: no building and un-classified buildings
            color = buildings_damage_colors[0]
            #print("This building has unknown damage class: "+u)     
        
        # Filling the shrunken polygon to add a border between close polygons
        fillPoly(blank_building, [ns_poly], (1, 1, 1))
        fillPoly(blank_damage, [ns_poly], color)
        # updating labels for eack iteration (= building)
        mask_img_buildings += blank_building
        mask_img_damage += blank_damage


    #print(np.count_nonzero(mask_img_damage.flatten()))
    # solving the problem of overlapping buildings --> zeros will be in the pixels where
    #there is no building and where buildings overlapped (pixels with values > 1)
    mask_img_buildings[mask_img_buildings > 1] = 0
    mask_img_buildings[mask_img_buildings == 1] = 1
    # multiplying the damage array and the previous array will leave values only where there are ones
    final_mask = mask_img_buildings * mask_img_damage
    return final_mask


In [26]:
# from XView2 baseline
def read_json(json_path):
    """
    :param json_path: path to load json from
    :returns: a python dictionary of json features
    """
    annotations = json.load(open(json_path))
    return annotations

In [27]:
# from XView2 baseline
def get_feature_info(feature):
    """
    :param feature: a python dictionary of json labels
    :returns: a list mapping of polygons contained in the image 
    """
    # Getting each polygon points from the json file and adding it to a dictionary of uid:polygons
    props = {}

    for feat in feature['features']['xy']:
        feat_shape = wkt.loads(feat['wkt'])
        coords = list(mapping(feat_shape)['coordinates'][0])
        props[feat['properties']['uid']] = (np.array(coords, np.int32))

    return props

In [28]:
#it creates a dictionary with the damage labels by building. Uses the same key that props dict.
def get_feature_damage(feature):
    """
    :param feature: a python dictionary of json labels
    :returns: a list mapping of damage labels contained in the post-disaster image
    """
    damage = {}

    for feat in feature['features']['xy']:
        try:
            damage_type = feat['properties']['subtype']
        except: # pre-disaster damage is default no-damage
            damage_type = "no-damage"
            print("no damage info in "+feat['properties']['uid'])
            continue  
        damage[feat['properties']['uid']] = damage_type

    return damage

In [29]:
# Modified from XView2 baseline. Now it includes information of the post-damage files

def mask_chips(json_path, images_directory, output_directory):
    """
    :param json_path: path to json files
    :param images_directory: path to images
    :param output_directory: path to dir where to store masks
    """
    # For each feature in the json we will create a separate mask
    # Getting all files in the directory provided for jsons
    jsons_pre = [j for j in next(walk(json_path))[2] if '_pre' in j]
    # After removing non-json items in dir (if any)
    for j in tqdm([j for j in jsons_pre if j.endswith('json')],
                  unit='poly',
                  leave=False):
        # Our chips start off in life as PNGs
        chip_image_id_pre = path.splitext(j)[0] + '.png'
        mask_file = path.splitext(j.replace('_pre', '_post'))[0]

        # Loading the per chip json pre-disaster
        j_full_path_pre = path.join(json_path, j)
        chip_json_pre = read_json(j_full_path_pre)
        
        #getting the name of the post-json
        j_post = j.replace('_pre', '_post')
        #print(j_post)
        
        # Loading the per chip json post-disaster
        j_full_path_post = path.join(json_path, j_post)
        chip_json_post = read_json(j_full_path_post)

        # Getting the full chip path, and loading the size dimensions (same for post)
        chip_file_pre = path.join(images_directory, chip_image_id_pre)
        chip_size = get_dimensions(chip_file_pre)

        # Reading in the polygons from the json file
        polys_pre = get_feature_info(chip_json_pre)
        polys_post = get_feature_damage(chip_json_post)

        # Getting a list of the polygons and saving masks as separate or single image files
        if len(polys_pre) > 0:
       
            
            masked_polys = mask_polygons_together_with_border(chip_size, polys_pre,polys_post, 2)
            
            save_one_mask(masked_polys, output_directory, mask_file)
            

#### 3. Use all functions to generate damage masks

In [30]:
# Modified from XView2 baseline

# Getting the list of the disaster types under the xBD directory
path_xBD='../data/data_by_disaster'
disasters = next(walk(path_xBD))[1]

for disaster in tqdm(disasters, desc='Masking', unit='disaster'):
    # Create the full path to the images, labels, and mask output directories
    image_dir = path.join(path_xBD, disaster, 'images')
    json_dir = path.join(path_xBD, disaster, 'labels')
    output_dir = path.join(path_xBD, disaster, 'masks_building_damage')

    if not path.isdir(image_dir):
        print(
            "Error, could not find image files in {}.\n\n"
            .format(image_dir),
            file=stderr)
        exit(2)

    if not path.isdir(json_dir):
        print(
            "Error, could not find labels in {}.\n\n"
            .format(json_dir),
            file=stderr)
        exit(3)

    if not path.isdir(output_dir):
        makedirs(output_dir)
    
    mask_chips(json_dir, image_dir, output_dir)

Masking: 100%|██████████| 9/9 [00:16<00:00,  1.79s/disaster]
